In [2]:
from prettytable import PrettyTable
from matplotlib import pyplot as plt
import numpy as np
import statistics
from collections import defaultdict

In [3]:
class TeamStat:
    def __init__(self, team_code, matches, won, lost, super_over, no_result, nrr, nrr_for, nrr_against, points, league_pos, final_pos):
        self.team_code = team_code
        self.matches = matches
        self.won = won
        self.lost = lost
        self.super_over = super_over
        self.no_result = no_result
        self.nrr = nrr
        self.nrr_for = nrr_for
        self.nrr_against = nrr_against
        self.points = points
        self.league_pos = league_pos
        self.final_pos = final_pos

In [4]:
class IplTournament:
    def __init__(self, year, edition, team_stats, total_teams):
        self.year = year
        self.edition = edition
        self.team_stats = team_stats
        self.total_teams = total_teams

In [5]:
# Constants

first_tournament_year = 2008
latest_tournament_year= 2022
col_team = 0
col_matches = 1
col_won = 2
col_lost = 3
col_super_over = 4
col_no_result = 5
col_nrr = 6
col_for = 7
col_against = 8
col_points = 9
col_league_pos = 10
col_final_pos = 11

In [6]:
# Global Data structure

team_name_map = dict()
ipl_tournaments = []
team_performance_map = dict()
team_pos_map = dict()
team_points_map = dict()
team_nrr_map = dict()

In [7]:
def create_team_name_map():
    import csv
    with open('../data/teams.csv') as teams_file:
        heading = next(teams_file)
        teams_file = csv.reader(teams_file)
        for row in teams_file:
            team_name_map[row[0]] = row[1]
    print("Total teams: ", len(team_name_map))
    print("\nTeam names: \n")
    print("\n".join(team_name_map.values()))

In [8]:
create_team_name_map()

Total teams:  15

Team names: 

Rajasthan Royals
Punjab Kings
Mumbai Indians
Chennai Superkings
Delhi Capitals / Delhi Daredevils
Kolkata Knightriders
Sunrisers Hyderabad
Deccan Chargers
Gujarat Titans
Lucknow Supergiants
Pune Warriors India
Gujarat Lions
Royal Challengers Bangalore
Rising Pune Supergiants
Kochi Tuskers


In [9]:
def populate_ipl_tournamnet_data():
    for year in range(first_tournament_year, latest_tournament_year + 1):
        filepath = '../data/' + str(year) + '.csv'
        with open(filepath) as tournament_file:
            heading = next(tournament_file)
            tournament_file = csv.reader(tournament_file)
            team_stats = []
            for row in tournament_file:
                team_stat = TeamStat(row[col_team], int(row[col_matches]), int(row[col_won]), int(row[col_lost]), int(row[col_super_over]), int(row[col_no_result]), float(row[col_nrr]), row[col_for], row[col_against], int(row[col_points]), int(row[col_league_pos]), int(row[col_final_pos]))
                team_stats.append(team_stat)
            tournament = IplTournament(year, year-first_tournament_year+1, team_stats, len(team_stats)) 
            ipl_tournaments.append(tournament)
            
def print_yoy_team_matches():
    table = PrettyTable(['Year', 'Teams', 'Matches per team'])
    for tournament in ipl_tournaments:
        table.add_row([tournament.year, tournament.total_teams, tournament.team_stats[0].matches])
    
    print("\nYear by Year team and matches count\n")
    print(table)
    

In [10]:
populate_ipl_tournamnet_data()
print_yoy_team_matches()

NameError: name 'csv' is not defined

In [ ]:
# Create a map team_performance_map where the key is team_code and value is a list of stats
# the order in the list is team_code, total_seasons, total_matches, won, loss, tie, nr, win_ratio, loss_ratio

def create_team_performance_map():
    for tournament in ipl_tournaments:
        for team_stat in tournament.team_stats:
            if team_stat.team_code in team_performance_map:
                team_performance = team_performance_map[team_stat.team_code]
                team_performance[1] = team_performance[1] + 1
                team_performance[2] += team_stat.matches
                team_performance[3] += team_stat.won
                team_performance[4] += team_stat.lost
                team_performance[5] += team_stat.super_over
                team_performance[6] += team_stat.no_result
            else :
                team_performance = [team_stat.team_code, 1, team_stat.matches, team_stat.won, team_stat.lost, team_stat.super_over, team_stat.no_result]
                team_performance_map[team_stat.team_code] =  team_performance

def print_team_performance():
    table = PrettyTable(['Team', 'Seasons', 'Matches', 'Wins', 'Lost', 'Super over', 'No result', 'Win %', 'Lost %'])
    team_performance_print_list = []
    for team_code in team_performance_map:
        team_performance = team_performance_map[team_code]
        win_percent =  round(team_performance[3]*100/team_performance[2], 2)
        loss_percent = round(team_performance[4]*100/team_performance[2], 2)
        team_performance.append(win_percent)
        team_performance.append(loss_percent)
        team_performance_print_list.append(team_performance)
    team_performance_print_list.sort(key = lambda row: (row[2], row[7], -row[8]), reverse = True)
    for team_performance_print in team_performance_print_list:
        table.add_row(team_performance_print)
    print("\nTeam performance\n")
    print(table)

In [ ]:
create_team_performance_map()
print_team_performance()

### Points table

In [ ]:
# Create a map team_pos_map where the key is team_code and value is a list of stats
# the order in the list is team_code, total_seasons, total_trophies, total_playoffs, total_finals, playoff ratio, finals ratio

def create_team_position_map():
    for tournament in ipl_tournaments:
        for team_stat in tournament.team_stats:
            if team_stat.team_code in team_pos_map:
                team_pos = team_pos_map[team_stat.team_code]
                team_pos[1] = team_pos[1] + 1
                team_pos[2] = team_pos[2] + (0, 1) [team_stat.final_pos == 1]
                team_pos[3] = team_pos[3] + (0, 1) [team_stat.league_pos <= 4]
                team_pos[4] = team_pos[4] + (0, 1) [team_stat.final_pos <= 2]
            else:
                team_pos = [team_stat.team_code, 1, (0, 1) [team_stat.final_pos == 1], (0, 1) [team_stat.league_pos <= 4], (0, 1) [team_stat.final_pos <= 2]]
                team_pos_map[team_stat.team_code] = team_pos

def print_team_position_stat():
    table = PrettyTable(['Team', 'Seasons', 'Trophies', 'Playoffs', 'Finals', 'Playoffs %', 'Finals %'])
    team_pos_print_list = []
    for team_code in team_pos_map:
        team_pos = team_pos_map[team_code]
        playoff_percent =  round(team_pos[3]*100/team_pos[1], 2)
        final_percent = round(team_pos[4]*100/team_pos[1], 2)
        team_pos.append(playoff_percent)
        team_pos.append(final_percent)
        team_pos_print_list.append(team_pos)
    team_pos_print_list.sort(key = lambda row: (row[2], row[4], row[3]), reverse = True)
    for team_pos_print in team_pos_print_list:
        table.add_row(team_pos_print)
    print("\nTeam position stat\n")
    print(table)

In [ ]:
create_team_position_map()
print_team_position_stat()

##### Team points each season

In [ ]:
def create_team_points_map():
    for tournament in ipl_tournaments:
        year = tournament.year
        for team_stat in tournament.team_stats:
            if team_stat.team_code in team_points_map:
                team_points = team_points_map[team_stat.team_code]
                team_points.append((year, team_stat.points))
                team_points_map[team_stat.team_code] =  team_points
            else:
                team_points_map[team_stat.team_code] = [(year, team_stat.points)]

In [ ]:
create_team_points_map()
print(team_points_map)

In [ ]:
def plot_points_yoy_graph():
    subplot_rows = 8
    subplot_cols = 2
    fig_number = 0
    # plt.subplots(8, 2, figsize=(20,50))

    # plt.tight_layout()
    for team_code in team_points_map:
        #subplot = axis[int(fig_number/subplot_cols), int (fig_number/subplot_rows)]
        plt.figure(figsize=[7, 5])
        fig_number = fig_number + 1
        team_points = team_points_map[team_code]
        x_cord = []
        y_cord = []
        for point in team_points:
            x_cord.append(point[0])
            y_cord.append(point[1])

        plt.xticks(np.arange(first_tournament_year-1, latest_tournament_year+1, 1))
        plt.xticks(rotation=90)
        plt.xlim(first_tournament_year-1, latest_tournament_year+1)

        plt.ylim(0, 28)
        plt.yticks(np.arange(0, 28, 2))

        plt.xlabel('Year')
        plt.ylabel('Points')
        plt.title(team_code)

        for x, y in zip(x_cord, y_cord):
            plt.text(x, y, '{}'.format(y))
        plt.plot(x_cord,y_cord,  marker='.', color='g')
        plt.show()

In [ ]:
plot_points_yoy_graph()

### Most competitive year

In [ ]:
def print_points_table(year):
    table = PrettyTable(['Team', 'Points'])
    index = year-first_tournament_year
    tournament = ipl_tournaments[index]
    [(table.add_row([team_stat.team_code, team_stat.points])) for team_stat in tournament.team_stats]
    print(table)

In [ ]:
def calculate_competition_each_year():
    table = PrettyTable(['Year', 'Std deviation in points'])
    max_std_dev = 0
    min_std_dev = 1000
    most_competitive_year = 0
    least_competitive_year = 0
    for tournament in ipl_tournaments:
        points = [ (team_stat.points) for team_stat in tournament.team_stats]
        std_dev = statistics.stdev(points)
        if std_dev < min_std_dev:
            min_std_dev, most_competitive_year = std_dev, tournament.year
        if std_dev > max_std_dev:
            max_std_dev, least_competitive_year = std_dev, tournament.year
        table.add_row([tournament.year, round(std_dev, 2)])
    print(table)

    print('Most competitive year: ', most_competitive_year)
    print_points_table(most_competitive_year)
    print('Least competitive year: ', least_competitive_year) 
    print_points_table(least_competitive_year)

In [ ]:
calculate_competition_each_year()

### Team misssing due to NRR

In [ ]:
def find_unlucky_tables():
    unlucky_teams = defaultdict(list)
    for tournament in ipl_tournaments:
        fourth_team_points = tournament.team_stats[3].points
        for i in range(4, tournament.total_teams):
            team_stat = tournament.team_stats[i]
            if team_stat.points == fourth_team_points:
                unlucky_teams[team_stat.team_code].append(tournament.year)
            else :
                break

    # print logic            
    table = PrettyTable(['Team', 'Year'])
    for team_code in unlucky_teams:
        table.add_row([team_code, unlucky_teams[team_code]])
    print(table)        

In [ ]:
find_unlucky_tables()

### Teams in playoff with negative NRR

In [ ]:
def find_playoff_teams_with_negative_nrr():
    teams_qualified_with_negative_nrr = defaultdict(list)
    min_nrr = 0.0
    team_with_min_nrr = ''
    year_min_nrr = 0
    league_pos_min_nrr = 10
    for tournament in ipl_tournaments:
        for i in range(0, 4):
            team_stat = tournament.team_stats[i]
            if team_stat.nrr < 0:
                teams_qualified_with_negative_nrr[team_stat.team_code].append(tournament.year)
            if team_stat.nrr < min_nrr:
                min_nrr = team_stat.nrr
                team_with_min_nrr = team_stat.team_code
                year_min_nrr = tournament.year
                league_pos_min_nrr = team_stat.league_pos


    # print logic            
    table = PrettyTable(['Team', 'Year'])
    for team_code in teams_qualified_with_negative_nrr:
        table.add_row([team_code, teams_qualified_with_negative_nrr[team_code]])
    print(table)

    table = PrettyTable(['Team', 'NRR', 'Year', 'League Pos'])
    table.add_row([team_with_min_nrr, min_nrr, year_min_nrr, league_pos_min_nrr])
    print(table)

In [ ]:
find_playoff_teams_with_negative_nrr()

In [ ]:
def create_nrr_map():
    for tournament in ipl_tournaments:
        year = tournament.year
        for team_stat in tournament.team_stats:
            if team_stat.team_code in team_nrr_map:
                team_nrr = team_nrr_map[team_stat.team_code]
                team_nrr.append((year, team_stat.nrr))
                team_nrr_map[team_stat.team_code] =  team_nrr
            else:
                team_nrr_map[team_stat.team_code] = [(year, team_stat.nrr)]

In [ ]:
create_nrr_map()
print(team_nrr_map)

In [ ]:
def plot_nrr_yoy_graph():
    subplot_rows = 8
    subplot_cols = 2
    fig_number = 0
    # plt.subplots(8, 2, figsize=(20,50))

    # plt.tight_layout()
    for team_code in team_nrr_map:
        #subplot = axis[int(fig_number/subplot_cols), int (fig_number/subplot_rows)]
        plt.figure(figsize=[15, 8])
        fig_number = fig_number + 1
        team_nrr = team_nrr_map[team_code]
        x_cord = []
        y_cord = []
        for point in team_nrr:
            x_cord.append(point[0])
            y_cord.append(point[1])

        plt.xticks(np.arange(first_tournament_year-1, latest_tournament_year+1, 1))
        plt.xticks(rotation=90)
        plt.xlim(first_tournament_year-1, latest_tournament_year+1)

        plt.ylim(-2, 2)
        plt.yticks(np.arange(-2, 2, 0.2))

        plt.xlabel('Year')
        plt.ylabel('NRR')
        plt.title(team_code)

        for x, y in zip(x_cord, y_cord):
            plt.text(x, y, '{}'.format(y))
        plt.plot(x_cord,y_cord,  marker='.', color='g')
        plt.show()

In [ ]:
plot_nrr_yoy_graph()




\
\
\



